In [1]:
import sys
import pathlib
import torch

import numpy as np

from sklearn.metrics import mean_squared_error as calc_mse

# Local
cwd = pathlib.Path().resolve()
src = cwd.parent.parent
root = src.parent
sys.path.append(str(src))

from models.mlp import MLP
from models.unet import UNet
from models.unet_source import UNet_source
from models.unet_mask import UNet_mask

from utils.watertopo import WaterTopo
from utils.utils import recursive_pred, mse_per_timestep, get_corner

%matplotlib inline

### Test 1
Load the simulations of test 1, create inputs and targets

In [2]:
grid_size = 64
channels = 2

sims = WaterTopo.load_simulations(str(root)+"/data/normalized_data/test1", sim_amount=20, number_grids=grid_size, use_augmented_data=True)

dur = len(sims[0].wd)

X   = np.zeros((len(sims), 1, 2, grid_size, grid_size))
Y0  = np.zeros((len(sims), dur, 1, grid_size, grid_size))
Y5  = Y0[:,::5+1,:,:,:]
Y10 = Y0[:,::10+1,:,:,:]

for i,sim in enumerate(sims):
    X[i,:,0,:,:] = sim.topography
    X[i,:,1,:,:] = sim.wd[0]

    Y0[i,:,0,:,:]  = sim.wd
    Y5[i,:,0,:,:]  = sim.implement_skips(5).wd
    Y10[i,:,0,:,:] = sim.implement_skips(10).wd

print("X: ", X.shape)
print("Y0: ", Y0.shape)
print("Y5: ", Y5.shape)
print("Y10: ", Y10.shape)


X:  (20, 1, 2, 64, 64)
Y0:  (20, 97, 1, 64, 64)
Y5:  (20, 17, 1, 64, 64)
Y10:  (20, 9, 1, 64, 64)


Multistep-models

In [3]:
# Load the model
model = UNet(2, [32, 64, 128], 96)
model_type = "unet_multistep"
model_name = "unet_32_64_128_orig_data80_multistep_skip0"
Y = Y0
skips = 0

model.load_state_dict(torch.load(str(src)+"/results/trained_models/" + model_type + "/" + model_name, 
                                 map_location="cpu"))

mse = np.zeros((len(sims), Y.shape[1]))

for i, sim in enumerate(sims):
    inputs = torch.tensor(X[i], dtype=torch.float32)
    outputs = model(inputs).squeeze()

    outputs = torch.cat((torch.tensor(sim.wd[0], dtype=torch.float32).unsqueeze(0), outputs), dim=0)
    
    mse[i,:] = mse_per_timestep(Y[i].squeeze(), outputs)

mse = np.mean(mse, axis=0)
t = np.arange(0, 97, skips+1)
mse = np.vstack([t, mse])

# np.savetxt(str(src)+"/results/error_accumulation/"+model_name, mse)

Auto-regressive models

In [14]:
# Load the model
model = UNet_source(2, 1)
model_type = "unet"
model_name = "unet_source_aug_skip5"
Y = Y10
skips = 10

model.load_state_dict(torch.load(str(src)+"/results/trained_models/" + model_type + "/" + model_name, 
                                 map_location="cpu"))
model.eval()

mse = np.zeros((len(sims), Y.shape[1]))

for i, sim in enumerate(sims):
    inputs = torch.tensor(X[i], dtype=torch.float32).squeeze()

    with torch.no_grad():
        outputs = recursive_pred(model, inputs, Y.shape[1]-1, include_first_timestep=True)
    
    mse[i,:] = mse_per_timestep(Y[i].squeeze(), outputs)

mse = np.mean(mse, axis=0)
t = np.arange(0, 97, skips+1)
mse = np.vstack([t, mse])

# np.savetxt(str(src)+"/results/error_accumulation/"+model_name, mse)

print(mse)

[[0.00000000e+00 1.10000000e+01 2.20000000e+01 3.30000000e+01
  4.40000000e+01 5.50000000e+01 6.60000000e+01 7.70000000e+01
  8.80000000e+01]
 [5.42474701e-24 2.20505564e-02 2.97099462e-02 3.81496206e-02
  4.63352532e-02 5.36467716e-02 5.97951938e-02 6.42383775e-02
  6.72610081e-02]]


In [19]:
# Load the model
model = UNet_mask(2, [32, 64], 1, 5)
model_type = "unet_mask"
model_name = "unet_32_64_orig_data80_skip5_hardmask5"
Y = Y5
skips = 5

model.load_state_dict(torch.load(str(src)+"/results/trained_models/" + model_type + "/" + model_name, 
                                 map_location="cpu"))
model.eval()

mse = np.zeros((len(sims), Y.shape[1]))

for i, sim in enumerate(sims):
    inputs = torch.tensor(X[i], dtype=torch.float32).squeeze()

    with torch.no_grad():
        outputs = recursive_pred(model, inputs, Y.shape[1]-1, include_first_timestep=True)
    
    mse[i,:] = mse_per_timestep(Y[i].squeeze(), outputs)

mse = np.mean(mse, axis=0)
t = np.arange(0, 97, skips+1)
mse = np.vstack([t, mse])

# np.savetxt(str(src)+"/results/error_accumulation/"+model_name, mse)

print(mse)

[[0.00000000e+00 6.00000000e+00 1.20000000e+01 1.80000000e+01
  2.40000000e+01 3.00000000e+01 3.60000000e+01 4.20000000e+01
  4.80000000e+01 5.40000000e+01 6.00000000e+01 6.60000000e+01
  7.20000000e+01 7.80000000e+01 8.40000000e+01 9.00000000e+01
  9.60000000e+01]
 [5.42474701e-24 1.56773795e-03 3.43045790e-03 6.09518038e-03
  9.66525755e-03 1.39607415e-02 1.84215745e-02 2.27112738e-02
  2.65715951e-02 3.03082432e-02 3.36909858e-02 3.62233176e-02
  3.81002499e-02 3.93075290e-02 3.98939746e-02 4.01820495e-02
  4.03193243e-02]]


MLPs:

In [27]:
# Load the model
model = MLP(2, 256, 1)
model_type = "mlp"
model_name = "mlp_corners_aug_data"

model.load_state_dict(torch.load(str(src)+"/results/trained_models/" + model_type + "/" + model_name, 
                                 map_location="cpu"))
model.eval()

for i, sim in enumerate(sims):
    inputs = np.zeros((grid_size, grid_size, channels))
    targets = np.zeros((grid_size, grid_size))

    inputs[:,:,0] = get_corner(sim.wd[0])
    inputs[:,:,1] = sim.topography
    targets[:,:,] = sim.return_timestep(-1)

    inputs = torch.tensor(inputs, dtype=torch.float32)

    with torch.no_grad():
        outputs = model(inputs)
    
    mse = calc_mse(targets, outputs.squeeze())

np.savetxt(str(src)+"/results/error_accumulation/"+model_name, np.array([0,mse]))
print(mse)

0.01768131961300556
